# MOLEARN ANALYSIS TUTORIAL

## Introduction

[molearn](https://github.com/Degiacomi-Lab/molearn) is a Python package streamlining the implementation of machine learning models trainable on protein conformational spaces. This tutorial is dedicated to demonstrating the first principles of analysing the performance of a trained neural network. Implementations of what described below is available in the `MolearnAnalysis` class. You can find out how to use this class in the [molearn_GUI.ipynb](molearn_GUI.ipynb) notebook.

This tutorial is divided in the following sections:

[1. Load neural network and training set](#ref1)
   - [1.1. The data](#ref11)
   - [1.2. Data loading](#ref12)
   - [1.3. Loading the neural network](#ref13)
   
   
[2. Projection into latent space and back](#ref2)
 
 
[3. Dataset analysis](#ref3)
   -  [3.1. RMSD (or MSE) of input vs autoencoded](#ref31)
   -  [3.2. DOPE score of input vs autoencoded](#ref32)
   
   
[4. Full latent space characterization](#ref4)
   - [4.1. Evaluate re-encoding error](#ref41)
   - [4.2. Evaluate DOPE score](#ref42)

To get started, let's load some packages!

In [ ]:
import sys, os, glob
from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim

import nglview as nv
import MDAnalysis as mda
import warnings
warnings.filterwarnings("ignore")

import modeller
from modeller import *
from modeller.scripts import complete_pdb

import biobox as bb

from tqdm import tqdm
#edit path as required for your computer (or remove, if you installed molearn via conda-forge)
sys.path.insert(0, f"C:{os.sep}Users{os.sep}xdzl45{os.sep}workspace{os.sep}molearn{os.sep}src") 

import molearn
from molearn.models.foldingnet import AutoEncoder
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## 1. Load neural network and training set <a class="anchor" id="ref1"></a>

### 1.1. the data <a class="anchor" id="ref11"></a>

MurD is a 47-kDa ATP-driven ligase responsible for the biosynthesis of a bacterial peptidoglycan precursor (UDP-N-acetylmuramoyl-L-alanyl-D-glutamate). When bound to its ligand, UDP-N-acetylmuramoyl-L-alanyl-D-alanine, MurD is stabilized in a closed conformation (PDB: [3UAG](https://www.rcsb.org/structure/3UAG)). In the absence of UDP-N-acetylmuramoyl-L-alanyl-D-alanine, MurD takes instead an open conformation (PDB: [1E0D](https://www.rcsb.org/structure/1E0D)). We have carried out molecular dynamics simulations of both states. For this tutorial, we will load a multi-PDB file containing a subset of conformations of both trajectories.

Let's start by defining the files containing the training set (variable `open_file` and `closed_file`), and test set (variable `test_set_file`).

In [ ]:
open_file = f'data{os.sep}MurD_open_selection.pdb'
closed_file = f'data{os.sep}MurD_closed_selection.pdb'
test_set_file = f'data{os.sep}MurD_closed_apo_selection.pdb'

The open_file and closed_file contain training set MD simulations of the closed and open states.

In [ ]:
universe_training = mda.Universe(open_file)
w = nv.show_mdanalysis(universe_training)
w

In [ ]:
universe_training = mda.Universe(closed_file)
w = nv.show_mdanalysis(universe_training)
w

By taking the closed conformation and manually removing the ligand, the protein switches to an open conformation. This is the simulation we will used as test set.

In [ ]:
universe_test = mda.Universe(test_set_file)
w = nv.show_mdanalysis(universe_test)
w

### 1.2. data loading <a class="anchor" id="ref12"></a>

We will now prepare the training set, transforming into a `torch.Tensor` making sure to extract from it only the atoms actually used during neural network training (CA, C, N, O, CB). The following cell yields a PDBData class with attributes:
 - `PDBData.dataset`, a pyTorch `Tensor` containing a desired subset of *normalized* atomic coordinates ready for submission to the neural network.
 - `PDBData.mean` and `PDBData.std`, the mean and standard deviation of the original input data, useful to rescale generated structures into atomic coordinates.
 - `PDBData.get_atominfo()` method that returns a list containing the names, resid, and resname of atoms in the training set (in the same order)
 - `PDBData.mol`, a  `biobox` instance corresponding to the coordinates in `PDBData.dataset`, useful as a PDB writer and offering useful helper functions, e.g. RMSD calculation.


In [ ]:
data = molearn.PDBData()
data.import_pdb(open_file)
data.import_pdb(closed_file) # this will append closed_file structures onto open_files
data.atomselect(atoms=['CA', 'C', 'CB', 'N', 'O'])
data.prepare_dataset()

training_set = data.dataset
stdval = data.std
meanval = data.mean
mol = data.mol
atom_names = data.get_atominfo()

`training_set` takes the form of a Nx3xM normalised `Tensor` object, where N is the number of examples, M the number of atoms, and 3 the x, y, z coordinates of each atom. This is the order of dimensions required by the neural network, a typical multiPDB has columns in the NxMx3 order.

In [ ]:
print(training_set.shape)

Let's now load the test set in the same way as the training set. This time, we will only retain the coordinates of atoms, normalised and ready to be submitted to the neural network (discarding all other outputs). The test set must contain the same atoms as the training set, in the same order. The two datasets should also be aligned, since *molearn* is not rototranslation invariant.

In [ ]:
test_data = molearn.PDBData()
test_data.import_pdb(test_set_file)
test_data.atomselect(atoms=data.atoms) # use same atoms as in data
test_data.std, test_data.mean = data.std, data.mean # use same std and mean as data
test_data.prepare_dataset()

test_set = data.dataset

### 1.3. loading the neural network <a class="anchor" id="ref13"></a>

We can now load the parameters of a trained neural network saved in the following file (variable `networkfile`):

In [ ]:
networkfile = f'data{os.sep}checkpoint_no_optimizer_state_dict_epoch167_loss0.003259085263643.ckpt'

Note that the parameters passed to the `AutoEncoder` costructor need to be the same as those used to build the trained neural network.

In [ ]:
checkpoint = torch.load(networkfile, map_location=device)

network = AutoEncoder(**checkpoint['network_kwargs'])
network.load_state_dict(checkpoint['model_state_dict'])

network.eval()

In [ ]:
print(next(network.parameters()))

## 2. Projection into latent space and back <a class="anchor" id="ref2"></a>

Let's start by projecting the whole training set into the latent space. Due to memory constraints we often can't simply encode or decode the entire dataset in one go like:
```
with torch.no_grad():
    z = network.encode(training_set.float())
```
We will need to do this in batches instead.

In [ ]:
BATCH_SIZE = 8

def batched_encode(dataset):
    # Preallocate space for the results to go
    z = torch.empty(dataset.shape[0], 2, 1, dtype=dataset.dtype, device = dataset.device)
    with torch.no_grad():
        for i in tqdm(range(0, z.shape[0], BATCH_SIZE)):
            z[i:i+BATCH_SIZE] = network.encode(dataset[i:i+BATCH_SIZE].float())
    return z

def batched_decode(latent_vector, n_atoms):
    decoded = torch.empty(latent_vector.shape[0], 3, n_atoms, dtype=latent_vector.dtype, device = latent_vector.device)
    with torch.no_grad():
        for i in tqdm(range(0, decoded.shape[0], BATCH_SIZE)):
            decoded[i:i+BATCH_SIZE] = network.decode(latent_vector[i:i+BATCH_SIZE].float())[:,:,:n_atoms]
    return decoded

In [ ]:
z = batched_encode(dataset = training_set.float())

`z` is a (Nx2x1) `Tensor` object, featuring the projection in the latent space of each of the N examples in the training set. If this notebook is running in a computer with a GPU, it will be stored in its memory. To get the data so we can plot it, we should move it back to the CPU, and remove one dimension.

In [ ]:
z_training = z.data.cpu().numpy()[:, :, 0]

`z` can be decoded back into structures that, hopefully, will closely resemble the training set. Note that we will have to clip the output to the size of dataset's degrees of freedom (an extra padding is present). As for the encoding part, while in principle it is possible to decode all the encoded training set in one shot as follows

```
with torch.no_grad():  
    decoded_training = network.decode(z)[:, :, :training_set.shape[2]]
```
it is often impossible to do so owing to memory constraints. Let's decode our dateset in batches!

In [ ]:
decoded_training = batched_decode(latent_vector = z, n_atoms = training_set.shape[2])

`decoded_training` is a now `Tensor` object stored in the GPU with the same dimensionality as the input `training_set`, and features normalised atomic coordinates. While it can already be directly compared with `training_set`, to obtain an actual protein structure we can observe, we should reorder the columns, move the data onto the CPU and "un-normalise" them.

In [ ]:
decoded_protein = decoded_training[:, :, :training_set.shape[2]].data.cpu().numpy().transpose(0, 2, 1)
print(decoded_protein.shape)

Ok, now let's repeat encoding and decoding with the test set!

In [ ]:
#the following commented lines encode and decode all the test set in one shot
#with torch.no_grad():
#    z = network.encode(test_set.float())
#    z_test = z.data.cpu().numpy()[:, :, 0]
#    decoded_test = network.decode(z)[:, :, :test_set.shape[2]]

z = batched_encode(dataset = test_set.float())
z_test = z.data.cpu().numpy()[:, :, 0]
decoded_test = batched_decode(latent_vector = z, n_atoms = test_set.shape[2])

## 3. dataset analysis <a class="anchor" id="ref3"></a>

### 3.1. RMSD (or MSE) of input vs autoencoded <a class="anchor" id="ref31"></a>

A way to evaluate the performance of the network, is to compare protein conformations before they are submitted to the neural network, and after they get encoded and decoded. Note that reporting such results for the training set only is poor practice, an independent test set should be submitted to evaluate the ability of the network to generalize beyond the examples provided for training. Let's calculate the MSE of both training and test sets!

In [ ]:
def get_error(network, dataset, meanval, stdval, align=False, mol=""):
    '''
    Calculate the reconstruction error of a dataset encoded and decoded by a trained neural network
    '''

    #z = network.encode(dataset.float())
    z = batched_encode(dataset.float())
    #decoded = network.decode(z)[:,:,:dataset.shape[2]]
    decoded = batched_decode(z, dataset.shape[2])
    err = []
    for i in tqdm(range(dataset.shape[0])):

        crd_ref = dataset[i].permute(1,0).unsqueeze(0).data.cpu().numpy()*stdval + meanval
        crd_mdl = decoded[i].permute(1,0).unsqueeze(0).data.cpu().numpy()[:, :dataset.shape[2]]*stdval + meanval #clip the padding of models  
        if align: # use Molecule Biobox class to calculate RMSD
            mol.coordinates = deepcopy(crd_ref)
            mol.set_current(0)
            mol.add_xyz(crd_mdl[0])
            rmsd = mol.rmsd(0, 1)
        else:
            rmsd = np.sqrt(np.sum((crd_ref.flatten()-crd_mdl.flatten())**2)/crd_mdl.shape[1]) # Cartesian L2 norm

        err.append(rmsd)

    return np.array(err)
    
rmsd_training = get_error(network, training_set, meanval, stdval)
rmsd_test = get_error(network, test_set, meanval, stdval)

Time to produce two nice violin plots, reporting on the error of training and test set. Typically the latter will be slightly higher than the former, though ideally this difference should be as small as possible, indicating that the neural network can generalize well to structures it has never seen.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.violinplot([rmsd_training, rmsd_test], showmeans=True)
ax.set_ylabel("RMSD / $\AA$")
ax.set_xticks([1, 2])
ax.set_xticklabels(["training set", "test set"]);

### 3.2. DOPE score of input vs autoencoded <a class="anchor" id="ref32"></a>

The DOPE score is a heuristic function used to estimate the energy of a protein conformation. It is used by software like Modeller to define whether a protein model is accurate. Here, we start by defining a function calculating the DOPE score of a PDB file.

In [ ]:
def dope_score(fname):
    env = Environ()
    env.libs.topology.read(file='$(LIB)/top_heav.lib')
    env.libs.parameters.read(file='$(LIB)/par.lib')
    mdl = complete_pdb(env, fname)
    atmsel = Selection(mdl.chains[0])
    score = atmsel.assess_dope()
    return score

A positive value for a DOPE score indicates that the distriution of atoms is not what one should expect from a correct protein conformation (i.e. the protein structure is bad). However, a negative DOPE score does not immediately tell us how good the structure is. To tell whether the models generated by the neural network are good, we will compare their DOPE score with the DOPE score of the input dataset, that we can assume being good. We now calculate the DOPE score of the training and test sets.

In [ ]:
def get_dope(network, dataset, meanval, stdval, mol):
    
    # set residues names with protonated histidines back to generic HIS name (needed by DOPE score function)
    testH = mol.data["resname"].values
    testH[testH == "HIE"] = "HIS"
    testH[testH == "HID"] = "HIS"
    mol.data["resname"] = testH

    #z = network.encode(dataset.float())
    z = batched_encode(dataset.float())
    #decoded = network.decode(z)[:,:,:dataset.shape[2]]
    decoded = batched_decode(z, dataset.shape[2])
    
    dope_dataset = []
    dope_decoded = []
    for i in tqdm(range(dataset.shape[0])):

        # calculate DOPE score of input dataset
        crd_ref = dataset[i].permute(1,0).unsqueeze(0).data.cpu().numpy()*stdval + meanval
        mol.coordinates = deepcopy(crd_ref)
        mol.write_pdb("tmp.pdb")
        s = dope_score("tmp.pdb")
        dope_dataset.append(s)

        # calculate DOPE score of decoded counterpart
        crd_mdl = decoded[i].permute(1,0).unsqueeze(0).data.cpu().numpy()[:, :dataset.shape[2]]*stdval + meanval  
        mol.coordinates = deepcopy(crd_mdl)
        mol.write_pdb("tmp.pdb")
        s = dope_score("tmp.pdb")
        dope_decoded.append(s)
    
    return dope_dataset, dope_decoded
 
dope_training, dope_training_decoded = get_dope(network, training_set, meanval, stdval, mol)
dope_test, dope_test_decoded = get_dope(network, test_set, meanval, stdval, mol)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
violins = ax.violinplot([dope_training, dope_training_decoded, dope_test, dope_test_decoded],
                        showmeans = True, showextrema = True)

ax.set_ylabel("DOPE / a.u.")
ax.set_xticks([1, 2, 3, 4])
ax.set_xticklabels(["training set", "training decoded", "test set", "test decoded"]);

colours=["red", "orange", "blue", "cyan"]
for i, violin in enumerate(violins['bodies']):
    violin.set_color(colours[i])
    violin.set_alpha(1)

for partname in ('cbars','cmins','cmaxes','cmeans'):
    vp = violins[partname]
    vp.set_edgecolor("k")
    vp.set_linewidth(1)

## 4. full latent space characterization <a class="anchor" id="ref4"></a>

In this section, we will grid sample the latent space. To this end, we start by identifying the region of interest defined by the coordinates of the training set projected into the latent space.

In [ ]:
samples = 50
bx = (np.max(z_training[:, 0]) - np.min(z_training[:, 0]))*0.1 # 10% margins on x-axis
by = (np.max(z_training[:, 1]) - np.min(z_training[:, 1]))*0.1 # 10% margins on y-axis
xvals = np.linspace(np.min(z_training[:, 0])-bx, np.max(z_training[:, 0])+bx, samples)
yvals = np.linspace(np.min(z_training[:, 1])-by, np.max(z_training[:, 1])+by, samples)

<div class="alert alert-block alert-warning">
<b>Warning:</b> depending on the sampling granularity, this operation can take a long time! Calculating the DOPE score of a 50x50 grid can take ~20 minutes. 
</div>

### 4.1. Evaluate re-encoding error <a class="anchor" id="ref41"></a>

Each position in the latent space is decoded, then encoded, the decoded yet again. We assess the drift in latent space upon this iteration and the difference in two consecutive decoded structures with L2 norms. This acts as a heuristic for the precision of the neural network: the smaller these numbers, the more precise the network. 

In [ ]:
surf_z = np.zeros((len(xvals), len(yvals))) # L2 norms in latent space ("drift")
surf_c = np.zeros((len(xvals), len(yvals))) # L2 norms in Cartesian space

with torch.no_grad():
    
    for x, i in tqdm(enumerate(xvals)):
        for y, j in enumerate(yvals):
    
            # take latent space coordinate (1) and decode it (2)
            z1 = torch.tensor([[[i,j]]]).float()
            s1 = network.decode(z1)[:,:,:training_set.shape[2]]
            
            # take the decoded structure, re-encode it (3) and then decode it (4)
            z2 = network.encode(s1)
            s2 = network.decode(z2)[:,:,:training_set.shape[2]]
            
            surf_z[x,y] = np.sum((z2.numpy().flatten()-z1.numpy().flatten())**2) # Latent space L2, i.e. (1) vs (3)
            surf_c[x,y] = np.sum((s2.numpy().flatten()-s1.numpy().flatten())**2) # Cartesian L2, i.e. (2) vs (4)
            
surf_c = np.sqrt(surf_c)
surf_z = np.sqrt(surf_z)

Plot error surfaces for L2 norms in the latent and Cartesian space, and compare the two. For visualisation purposes, we take the log of both quantities.

In [ ]:
X, Y = np.meshgrid(xvals, yvals)

fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(1, 3, 1)
ax.set_title("Latent space L2")
ax.pcolormesh(X, Y, np.log(surf_z))

ax = fig.add_subplot(1, 3, 2)
ax.set_title("3D space L2")
ax.pcolormesh(X, Y, np.log(surf_c))

ax = fig.add_subplot(1, 3, 3)
ax.set_xlabel("Latent space L2")
ax.set_ylabel("3D space L2")
ax.plot(surf_z.flatten(), surf_c.flatten(), ".", alpha=0.2);

You will notice that, although in general large drift corresponds to large RMSD, there is not a 1:1 relationship between the two quantities. This is because the latent space is not linear: the same magnitude of displacement in two different regions of the latent space may be associated to displacements of different magnitude in the 3D space. You may see several trends, overlayed, if the neural network has been trained with a collection of different molecular dynamics simulations projecting in different regions of the latent space.

### 4.2. Evaluate DOPE score <a class="anchor" id="ref42"></a>

We will now calculate the DOPE score of every conformation decoded from the latent space.

In [ ]:
surf_dope = np.zeros((len(xvals), len(yvals)))

with torch.no_grad():
    
    for x, i in tqdm(enumerate(xvals)):
        for y, j in enumerate(yvals):
    
            # take latent space coordinate (1) and decode it (2)
            z1 = torch.tensor([[[i,j]]]).float()
            s1 = network.decode(z1)[:,:,:training_set.shape[2]]
            
            crd_mdl = s1[0].permute(1,0).unsqueeze(0).data.cpu().numpy()[:, :training_set.shape[2]]*stdval + meanval  
            mol.coordinates = deepcopy(crd_mdl)
            mol.write_pdb("tmp.pdb")
            s = dope_score("tmp.pdb")
            surf_dope[x,y] = s

In [ ]:
x, y = np.meshgrid(xvals, yvals)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel("Latent vector 1 / a.u.")
ax.set_ylabel("Latent vector 2 / a.u.")
c = ax.pcolormesh(x, y, surf_dope.T)
#ax.plot(z_training[:, 0], z_training[:, 1], color="white", alpha=0.5)
cbar = fig.colorbar(c, ax=ax)
cbar.set_label("DOPE score / a.u.");

This is the end of this overview. The methods described here are implemented in `MolearnAnalysis.py`. Have a look at the Jupyter notebook [molearn_GUI.ipynb](molearn_GUI.ipynb) to see it in action!

---